In [ ]:
# Análise Exploratória de Dados (EDA) - Detecção de Ataques de Rede

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("📊 Iniciando Análise Exploratória de Dados")

In [ ]:
# 1. CARREGAMENTO E VISÃO GERAL DOS DADOS

# Carregamento dos dados processados
df = pd.read_csv('../data/processed/flows.csv')

print(f"✅ Dados carregados com sucesso!")
print(f"📈 Dimensões do dataset: {df.shape[0]} fluxos, {df.shape[1]} features")
print(f"🔍 Primeiras linhas:")
display(df.head())

In [ ]:
# 2. ESTRUTURA E TIPOS DE DADOS

print("🔬 Informações sobre os dados:")
print(f"📊 Formato: {df.shape}")
print(f"💾 Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print()

print("📋 Tipos de dados:")
display(df.dtypes.to_frame('Tipo'))
print()

print("❌ Valores ausentes:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Valores Ausentes': missing,
    'Percentual (%)': missing_pct
}).round(2)
display(missing_df[missing_df['Valores Ausentes'] > 0])

In [ ]:
# 3. ESTATÍSTICAS DESCRITIVAS

print("📈 Resumo estatístico das features numéricas:")
numeric_cols = ['bytes', 'pkts', 'duration', 'iat_mean', 'iat_std']
desc_stats = df[numeric_cols + ['label']].describe()
display(desc_stats.round(4))

print()
print("🎯 Estatísticas por classe (Normal vs Ataque):")
stats_by_label = df.groupby('label')[numeric_cols].describe()
display(stats_by_label.round(4))

In [ ]:
# 4. ANÁLISE DE BALANCEAMENTO DAS CLASSES

print("⚖️ Distribuição das classes:")
label_counts = df['label'].value_counts().sort_index()
label_props = df['label'].value_counts(normalize=True).sort_index()

balance_df = pd.DataFrame({
    'Classe': ['Normal (0)', 'Ataque (1)'],
    'Quantidade': label_counts.values,
    'Proporção (%)': (label_props.values * 100).round(2)
})

display(balance_df)

# Visualização
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Gráfico de barras
balance_df.plot(x='Classe', y='Quantidade', kind='bar', ax=ax1, color=['skyblue', 'salmon'])
ax1.set_title('📊 Distribuição Absoluta das Classes')
ax1.set_xlabel('Classe')
ax1.set_ylabel('Número de Fluxos')
ax1.tick_params(axis='x', rotation=0)

# Gráfico de pizza
ax2.pie(balance_df['Quantidade'], labels=balance_df['Classe'], autopct='%1.1f%%', 
        colors=['skyblue', 'salmon'], startangle=90)
ax2.set_title('🥧 Proporção das Classes')

plt.tight_layout()
plt.show()

# Análise de balanceamento
ratio = label_counts.iloc[1] / label_counts.iloc[0]
print(f"📊 Proporção Ataque/Normal: {ratio:.3f}")
if ratio < 0.1:
    print("⚠️  Dataset muito desbalanceado - considere técnicas de balanceamento")
elif ratio < 0.5:
    print("⚠️  Dataset moderadamente desbalanceado")
else:
    print("✅ Dataset relativamente balanceado")

In [ ]:
# 5. DISTRIBUIÇÕES UNIVARIADAS DAS FEATURES

print("📊 Analisando distribuições das features numéricas...")

fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.ravel()

for i, col in enumerate(numeric_cols):
    # Histograma com curva de densidade
    axes[i].hist(df[col], bins=50, alpha=0.7, density=True, color='skyblue', edgecolor='black')
    
    # Estatísticas da distribuição
    mean_val = df[col].mean()
    median_val = df[col].median()
    std_val = df[col].std()
    
    # Linhas de referência
    axes[i].axvline(mean_val, color='red', linestyle='--', label=f'Média: {mean_val:.2f}')
    axes[i].axvline(median_val, color='green', linestyle='--', label=f'Mediana: {median_val:.2f}')
    
    axes[i].set_title(f'📈 Distribuição de {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Densidade')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)
    
    # Teste de normalidade
    _, p_value = stats.normaltest(df[col].dropna())
    normality = "Normal" if p_value > 0.05 else "Não Normal"
    axes[i].text(0.02, 0.98, f'Normalidade: {normality}\nSkew: {df[col].skew():.2f}', 
                transform=axes[i].transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

# Remove o subplot extra
if len(numeric_cols) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

print("\n🔍 Interpretação das distribuições:")
for col in numeric_cols:
    skew = df[col].skew()
    print(f"• {col}: Assimetria = {skew:.2f} ", end="")
    if abs(skew) < 0.5:
        print("(distribuição aproximadamente simétrica)")
    elif abs(skew) < 1:
        print("(distribuição moderadamente assimétrica)")
    else:
        print("(distribuição altamente assimétrica)")

In [ ]:
# 6. ANÁLISE COMPARATIVA POR CLASSE (NORMAL VS ATAQUE)

print("🎯 Comparando distribuições entre classes Normal e Ataque...")

fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.ravel()

for i, col in enumerate(numeric_cols):
    # Boxplot comparativo
    box_data = [df[df['label'] == 0][col].dropna(), df[df['label'] == 1][col].dropna()]
    box = axes[i].boxplot(box_data, labels=['Normal', 'Ataque'], patch_artist=True)
    
    # Colorir as caixas
    colors = ['lightblue', 'lightcoral']
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
    
    axes[i].set_title(f'📦 {col} por Classe')
    axes[i].set_ylabel(col)
    axes[i].grid(True, alpha=0.3)
    
    # Estatísticas comparativas
    normal_mean = df[df['label'] == 0][col].mean()
    attack_mean = df[df['label'] == 1][col].mean()
    
    # Teste t para diferença de médias
    _, p_value = stats.ttest_ind(
        df[df['label'] == 0][col].dropna(), 
        df[df['label'] == 1][col].dropna()
    )
    
    significance = "Significativa" if p_value < 0.05 else "Não Significativa"
    
    axes[i].text(0.02, 0.98, 
                f'Normal: {normal_mean:.2f}\nAtaque: {attack_mean:.2f}\nDif: {significance}', 
                transform=axes[i].transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

# Remove o subplot extra
if len(numeric_cols) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

# Análise quantitativa das diferenças
print("\n📊 Resumo das diferenças entre classes:")
comparison_stats = []

for col in numeric_cols:
    normal_data = df[df['label'] == 0][col]
    attack_data = df[df['label'] == 1][col]
    
    _, p_value = stats.ttest_ind(normal_data.dropna(), attack_data.dropna())
    effect_size = (attack_data.mean() - normal_data.mean()) / df[col].std()
    
    comparison_stats.append({
        'Feature': col,
        'Normal_Mean': normal_data.mean(),
        'Attack_Mean': attack_data.mean(),
        'Diferença_%': ((attack_data.mean() - normal_data.mean()) / normal_data.mean() * 100),
        'P_Value': p_value,
        'Effect_Size': effect_size
    })

comparison_df = pd.DataFrame(comparison_stats)
display(comparison_df.round(4))

print("\n🔍 Features mais discriminativas (por tamanho do efeito):")
top_features = comparison_df.reindex(comparison_df['Effect_Size'].abs().sort_values(ascending=False).index)
for _, row in top_features.head(3).iterrows():
    print(f"• {row['Feature']}: Effect Size = {row['Effect_Size']:.3f}")

In [ ]:
# 7. ANÁLISE DE CORRELAÇÕES

print("🔗 Analisando correlações entre features...")

# Matriz de correlação
corr_matrix = df[numeric_cols].corr()

# Visualização da matriz de correlação
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Heatmap principal
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='RdBu_r', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8}, ax=ax1)
ax1.set_title('🌡️ Matriz de Correlação (Pearson)')

# Heatmap apenas das correlações fortes (|r| > 0.5)
strong_corr = corr_matrix.copy()
strong_corr[abs(strong_corr) < 0.5] = 0
sns.heatmap(strong_corr, annot=True, fmt='.3f', cmap='RdBu_r', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8}, ax=ax2)
ax2.set_title('🔥 Correlações Fortes (|r| ≥ 0.5)')

plt.tight_layout()
plt.show()

# Identificar correlações mais fortes
print("\n📈 Correlações mais fortes entre features:")
# Criar matriz triangular superior para evitar duplicatas
upper_triangle = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
correlation_pairs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if upper_triangle[i, j]:
            feature1 = corr_matrix.columns[i]
            feature2 = corr_matrix.columns[j]
            correlation = corr_matrix.iloc[i, j]
            correlation_pairs.append({
                'Feature_1': feature1,
                'Feature_2': feature2,
                'Correlação': correlation,
                'Magnitude': abs(correlation)
            })

correlation_df = pd.DataFrame(correlation_pairs)
correlation_df = correlation_df.sort_values('Magnitude', ascending=False)

print("Top 5 correlações mais fortes:")
display(correlation_df.head().round(3))

# Interpretação das correlações
print("\n🔍 Interpretação das correlações:")
for _, row in correlation_df.head(3).iterrows():
    corr_val = row['Correlação']
    if abs(corr_val) >= 0.7:
        strength = "muito forte"
    elif abs(corr_val) >= 0.5:
        strength = "forte"
    elif abs(corr_val) >= 0.3:
        strength = "moderada"
    else:
        strength = "fraca"
    
    direction = "positiva" if corr_val > 0 else "negativa"
    print(f"• {row['Feature_1']} ↔ {row['Feature_2']}: Correlação {strength} {direction} ({corr_val:.3f})")

# Análise de multicolinearidade
print(f"\n⚠️ Possíveis problemas de multicolinearidade (|r| > 0.8):")
high_corr = correlation_df[correlation_df['Magnitude'] > 0.8]
if len(high_corr) > 0:
    for _, row in high_corr.iterrows():
        print(f"• {row['Feature_1']} ↔ {row['Feature_2']}: {row['Correlação']:.3f}")
else:
    print("✅ Nenhuma correlação extremamente alta detectada")

In [ ]:
# 8. ANÁLISE BIVARIADA - SCATTER PLOTS

print("🎨 Criando scatter plots para visualizar relações entre features...")

# Pairplot com distinção por classe
g = sns.pairplot(data=df[numeric_cols + ['label']], 
                 hue='label', 
                 plot_kws={'alpha': 0.6, 's': 30},
                 diag_kind='hist',
                 palette=['skyblue', 'salmon'])

# Personalizar o plot
g.fig.suptitle('🔍 Análise Bivariada - Relações entre Features por Classe', 
               y=1.02, fontsize=16, fontweight='bold')

# Adicionar legendas personalizadas
for ax in g.axes.flat:
    if ax.legend_:
        ax.legend(labels=['Normal', 'Ataque'], loc='best')

plt.show()

# Análise de separabilidade das classes
print("\n🎯 Análise de separabilidade entre classes:")
print("Esta visualização ajuda a identificar:")
print("• Features que separam bem as classes (pontos bem agrupados por cor)")
print("• Relações lineares ou não-lineares entre features")
print("• Possíveis clusters nos dados")
print("• Outliers que podem afetar o modelo")

# Scatter plots individuais mais detalhados para as top 3 correlações
if len(correlation_df) >= 3:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for i in range(3):
        row = correlation_df.iloc[i]
        feature1, feature2 = row['Feature_1'], row['Feature_2']
        
        # Scatter plot colorido por classe
        normal_data = df[df['label'] == 0]
        attack_data = df[df['label'] == 1]
        
        axes[i].scatter(normal_data[feature1], normal_data[feature2], 
                       alpha=0.6, c='skyblue', label='Normal', s=30)
        axes[i].scatter(attack_data[feature1], attack_data[feature2], 
                       alpha=0.6, c='salmon', label='Ataque', s=30)
        
        axes[i].set_xlabel(feature1)
        axes[i].set_ylabel(feature2)
        axes[i].set_title(f'{feature1} vs {feature2}\n(r = {row["Correlação"]:.3f})')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 9. DETECÇÃO E ANÁLISE DE OUTLIERS

print("🔍 Detectando e analisando outliers nos dados...")

def detect_outliers_iqr(data, column):
    """Detecta outliers usando o método IQR (Interquartile Range)"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Análise de outliers por feature
outlier_summary = []
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.ravel()

for i, col in enumerate(numeric_cols):
    outliers, lower, upper = detect_outliers_iqr(df, col)
    outlier_count = len(outliers)
    outlier_pct = (outlier_count / len(df)) * 100
    
    # Armazenar informações do outlier
    outlier_summary.append({
        'Feature': col,
        'Total_Outliers': outlier_count,
        'Percentual_Outliers': outlier_pct,
        'Normal_Outliers': len(outliers[outliers['label'] == 0]),
        'Attack_Outliers': len(outliers[outliers['label'] == 1]),
        'Lower_Bound': lower,
        'Upper_Bound': upper
    })
    
    # Visualização
    axes[i].boxplot([df[df['label'] == 0][col], df[df['label'] == 1][col]], 
                   labels=['Normal', 'Ataque'], patch_artist=True)
    
    # Destacar outliers
    normal_outliers = outliers[outliers['label'] == 0][col]
    attack_outliers = outliers[outliers['label'] == 1][col]
    
    if len(normal_outliers) > 0:
        axes[i].scatter([1] * len(normal_outliers), normal_outliers, 
                       color='blue', alpha=0.6, s=20, label=f'Outliers Normal ({len(normal_outliers)})')
    if len(attack_outliers) > 0:
        axes[i].scatter([2] * len(attack_outliers), attack_outliers, 
                       color='red', alpha=0.6, s=20, label=f'Outliers Ataque ({len(attack_outliers)})')
    
    axes[i].set_title(f'📦 {col}\nOutliers: {outlier_count} ({outlier_pct:.1f}%)')
    axes[i].grid(True, alpha=0.3)
    if len(normal_outliers) > 0 or len(attack_outliers) > 0:
        axes[i].legend()

# Remove subplot extra
if len(numeric_cols) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

# Resumo dos outliers
outlier_df = pd.DataFrame(outlier_summary)
print("\n📊 Resumo de Outliers por Feature:")
display(outlier_df.round(2))

# Análise mais detalhada
print("\n🔍 Análise detalhada dos outliers:")
total_outliers = outlier_df['Total_Outliers'].sum()
print(f"• Total de outliers detectados: {total_outliers}")
print(f"• Percentual médio de outliers: {outlier_df['Percentual_Outliers'].mean():.2f}%")

most_outliers = outlier_df.loc[outlier_df['Total_Outliers'].idxmax()]
print(f"• Feature com mais outliers: {most_outliers['Feature']} ({most_outliers['Total_Outliers']} outliers)")

# Distribuição de outliers por classe
total_normal_outliers = outlier_df['Normal_Outliers'].sum()
total_attack_outliers = outlier_df['Attack_Outliers'].sum()
print(f"• Outliers em tráfego normal: {total_normal_outliers}")
print(f"• Outliers em tráfego de ataque: {total_attack_outliers}")

# Análise de outliers extremos (Z-score > 3)
print("\n⚠️ Outliers extremos (Z-score > 3):")
extreme_outliers_found = False

for col in numeric_cols:
    z_scores = np.abs(stats.zscore(df[col].dropna()))
    extreme_outliers = df[z_scores > 3]
    
    if len(extreme_outliers) > 0:
        extreme_outliers_found = True
        normal_extreme = len(extreme_outliers[extreme_outliers['label'] == 0])
        attack_extreme = len(extreme_outliers[extreme_outliers['label'] == 1])
        print(f"• {col}: {len(extreme_outliers)} outliers extremos (Normal: {normal_extreme}, Ataque: {attack_extreme})")

if not extreme_outliers_found:
    print("✅ Nenhum outlier extremo detectado")

print("\n💡 Recomendações:")
high_outlier_features = outlier_df[outlier_df['Percentual_Outliers'] > 10]
if len(high_outlier_features) > 0:
    print("⚠️ Features com muitos outliers (>10%):")
    for _, row in high_outlier_features.iterrows():
        print(f"   • {row['Feature']}: {row['Percentual_Outliers']:.1f}% - considere transformação ou remoção")
else:
    print("✅ Percentual de outliers aceitável em todas as features")

# 10. CONCLUSÕES E INSIGHTS

## 🎯 Principais Descobertas da EDA

### Estrutura dos Dados
- **Tamanho do dataset**: Número total de fluxos e features analisadas
- **Qualidade dos dados**: Presença de valores ausentes e tipos de dados
- **Balanceamento**: Distribuição entre tráfego normal e de ataque

### Features Mais Discriminativas
- **Bytes**: Diferenças no volume de dados transferidos
- **Pacotes**: Número de pacotes por fluxo
- **Duração**: Tempo de duração dos fluxos
- **Inter-arrival times**: Padrões temporais entre pacotes

### Padrões Identificados
- **Tráfego Normal**: Características típicas observadas
- **Tráfego de Ataque**: Padrões anômalos detectados
- **Correlações**: Relações importantes entre features

### Outliers e Anomalias
- **Prevalência**: Percentual de outliers em cada feature
- **Distribuição**: Como os outliers se distribuem entre as classes
- **Impacto**: Possível influência nos modelos de ML

## 📈 Recomendações para Modelagem

1. **Pré-processamento**:
   - Normalização/padronização das features
   - Tratamento de outliers (remoção ou transformação)
   - Engenharia de features baseada nas correlações encontradas

2. **Seleção de Features**:
   - Priorizar features com maior poder discriminativo
   - Considerar remoção de features altamente correlacionadas

3. **Validação**:
   - Atenção ao balanceamento das classes
   - Uso de métricas apropriadas para datasets desbalanceados
   - Validação cruzada estratificada

4. **Modelos Sugeridos**:
   - Random Forest (robusto a outliers)
   - SVM (bom para separação de classes)
   - Gradient Boosting (captura relações complexas)

## 🔍 Próximos Passos

1. **Feature Engineering**: Criar novas features baseadas nos insights
2. **Modelagem**: Treinar e comparar diferentes algoritmos
3. **Validação**: Avaliar performance com métricas apropriadas
4. **Interpretabilidade**: Analisar importância das features nos modelos